This notebook creates the csv for the transaction table

In [13]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [14]:
# Read in transaction_items csv: 

trans_item = pd.read_csv("/home/srodier/repos/jugos_rodier/csvs/transaction_items.csv")

trans_item.head()

,transaction_item_id,trans_id,item_id,item_price,quantity
0,0,trans0,01hmsw03,7.23,1
1,1,trans0,02crep02,5.00,1
2,2,trans0,02brln02,3.77,1
3,3,trans0,04wtpe02,3.25,1
4,4,trans0,05fry03,3.00,1


In [15]:
# drop unecessary columns: 

trans_item.drop(['transaction_item_id', 'item_id'], axis = 1, inplace = True)

trans_item.head(15)

,trans_id,item_price,quantity
0,trans0,7.23,1
1,trans0,5.00,1
2,trans0,3.77,1
3,trans0,3.25,1
4,trans0,3.00,1
5,trans0,2.50,1
6,trans1,4.60,1
7,trans1,3.77,2
8,trans1,5.50,1
9,trans2,6.00,1


In [16]:
# ensure there are no null values and that data types are correct:

trans_item.info() # trans_id will be converted to category type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28791 entries, 0 to 28790
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   trans_id    28791 non-null  object 
 1   item_price  28791 non-null  float64
 2   quantity    28791 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 674.9+ KB


In [17]:
# multiply item_price by 2 if quantity is = 2 so that summing transaction total is easier:
for i in range(0, len(trans_item)):
    if trans_item.loc[i,'quantity'] == 2: 
        trans_item.loc[i, 'item_price'] = round(trans_item.loc[i, 'item_price'] * 2,2)
    else: 
        continue

trans_item.head(15)

,trans_id,item_price,quantity
0,trans0,7.23,1
1,trans0,5.00,1
2,trans0,3.77,1
3,trans0,3.25,1
4,trans0,3.00,1
5,trans0,2.50,1
6,trans1,4.60,1
7,trans1,7.54,2
8,trans1,5.50,1
9,trans2,6.00,1


In [18]:
# drop unecessary column: 

trans_item.drop('quantity', axis = 1, inplace = True)

    # Specify categories for trans_item.trans_id: 

trans_item['trans_id'] = pd.Categorical(trans_item['trans_id'], 
                                        categories= trans_item['trans_id'].unique(),
                                        ordered= True)

# Create transaction_total column: 

    # group by trans_id: 

trans = trans_item.groupby('trans_id')['item_price'].sum().reset_index()    

    # rename item_price column: 

trans = trans.rename(columns = {'item_price': 'transaction_total'}, copy = False)

trans.head(10)

/tmp/ipykernel_37124/3268113967.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trans = trans_item.groupby('trans_id')['item_price'].sum().reset_index()


,trans_id,transaction_total
0,trans0,24.75
1,trans1,17.64
2,trans2,20.27
3,trans3,44.75
4,trans4,15.77
5,trans5,16.62
6,trans6,20.25
7,trans7,9.52
8,trans8,24.77
9,trans9,15.04


In [19]:
# Assign random customer ids to each transaction: 
    # I will sample with replacement for simplicity's sake: 

customer = pd.read_csv("/home/srodier/repos/jugos_rodier/csvs/customer_info.csv")

customers = random.choices(customer['customer_id'], k = len(trans))

trans['customer_id'] = customers

trans.head()

,trans_id,transaction_total,customer_id
0,trans0,24.75,2410
1,trans1,17.64,317
2,trans2,20.27,649
3,trans3,44.75,591
4,trans4,15.77,1185


In [20]:
# assign a random time of day to each transaction: 

    # Create time of day column: 

trans['time_of_day'] = datetime.strptime('09:00:00', '%H:%M:%S').time()


    # Set the jugos rodier business times
start_time = datetime.strptime('09:00:00', '%H:%M:%S')
end_time = datetime.strptime('20:00:00', '%H:%M:%S')
    
    # Calculate the range of time (in seconds)
time_range = end_time - start_time

for i in range(len(trans)):

        # Generate a random number of seconds within the time range
    random_int = float(random.randint(0, int(time_range.total_seconds())))

    random_seconds = timedelta(seconds = random_int)
        # Add the random number of seconds to the start time

    random_time = (start_time + random_seconds)

    random_time = random_time.time()
    trans.loc[i, 'time_of_day'] = random_time

# make sure everything looks good:

trans.sample(20)

,trans_id,transaction_total,customer_id,time_of_day
665,trans665,22.10,983,12:28:53
2827,trans2828,14.00,1167,16:26:05
4771,trans4772,26.27,2097,15:01:16
2596,trans2597,27.77,1058,10:04:28
2131,trans2132,16.00,1989,19:09:20
4291,trans4292,23.60,990,11:41:57
2879,trans2880,21.50,1016,14:29:33
1222,trans1222,21.50,2151,16:01:11
1342,trans1343,28.27,182,18:14:58
465,trans465,25.50,294,09:13:51


In [21]:
# write to csv: 

trans.to_csv("/home/srodier/repos/jugos_rodier/csvs/transaction.csv", index = False)